# ZMRRELPO ANALYSIS

## 0.0 Notes and Explainations

This notebook uses SQL to query the FinanceDa Dev Server to extract the information contained in the t-code ZMRRELPO

### 0.1 Environment setup

This workbook utilizes the py37_benzene environment which can be installed via the Anaconda Prompt from your local repo sync by running:
> conda env create -f py37_benzene.yml


### 0.2 Updating

Any changes to the assembly of the final df need to be migrated to the Python function file at:

> Benzene Forecasting -- Scripts -- Integrated Modeling -- ZMRRELPO_SQL_Import

## 1.0 Prepare Workspace

### 1.1 Import Libraries and configure runtime parameters

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import pyodbc
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import os

# Configure Libraries
pio.renderers.default = "notebook_connected"

### 1.2 Configure SQL Connection

In [2]:
# Setup SQL Connection
username = 'ben.anderson@lyondellbasell.com'
driver = 'ODBC Driver 17 for SQL Server'
Authentication= 'ActiveDirectoryInteractive'

# Setup SQL Server
FDA_PRD_server = 'lyb-sql-prddacore-002.bed79ae4ef8b.database.windows.net'
FDA_PRD_database = 'Source'

FDA_DEV_server = 'lyb-sql-devdacore-002.5bff9fcb8330.database.windows.net'
FDA_DEV_database = 'FinanceDa'

In [3]:
# Establish pyodbc connection to servers
if 'ODBC Driver 17 for SQL Server' in pyodbc.drivers():
    
    # Connect to PRD FinanceDa Database
    try:
        Fin_PRD_DA = pyodbc.connect('DRIVER='+driver+
                    ';SERVER='+FDA_PRD_server+
                    ';PORT=1433;DATABASE='+FDA_PRD_database+
                    ';UID='+username+
                    ';AUTHENTICATION='+Authentication
                    )
        print("FinanceDA PRD\t\tEstablished and ready to be used")
    except:
        print("FinanceDA PRD\t\tFailed")

    # Connect to DEV FinanceDa Database
    try:
        Fin_DEV_DA = pyodbc.connect('DRIVER='+driver+
                    ';SERVER='+FDA_DEV_server+
                    ';PORT=1433;DATABASE='+FDA_DEV_database+
                    ';UID='+username+
                    ';AUTHENTICATION='+Authentication
                    )
        print("FinanceDA DEV\t\tEstablished and ready to be used")
    except:
        print("FinanceDA DEV\t\tFailed")
    
    
else:
    print("Necessary Driver (ODBC Driver 17 for SQL Server), not installed, cannot connect to database ")

FinanceDA PRD		Established and ready to be used
FinanceDA DEV		Established and ready to be used


## 2. 0 Load Data

### 2.1 Define SQL Queries

Each query is defined as a variable that contains multi-line text surrounded by """

In [4]:
# Daily Futures Commodities List
ZMRRELPO_Query = """
/****** Extract all rows from Custom view that provides ZMRRELPO clone as a SQL query  ******/
SELECT [EBELN]
      ,[EBELP]
      ,[BSART]
      ,[LIFNR]
      ,[NAME1]
      ,[EKORG]
      ,[EKGRP]
      ,[BEDAT]
      ,[BSTYP]
      ,[WERKS]
      ,[MATKL]
      ,[NETWR]
      ,[PSTYP]
      ,[MTART]
      ,[ELIKZ]
      ,[ERNAM]
      ,[MENGE]
      ,[NETPR]
      ,[NETPR2]
      ,[MEINS]
      ,[WAERS]
      ,[EMATN]
  FROM [FinanceDa].[dbo].[ZMRRELPO_benzene]
"""

### 2.2 Execute SQL Queries

In [5]:
# Extract Query into pd.DataFrame

df = pd.read_sql(ZMRRELPO_Query, Fin_DEV_DA)
df.shape

(963, 22)

## 3. 0 Explore Data

### 3.1 Pandas Profiling Report

In [6]:
#Profile = ProfileReport(df, minimal=True)
#Profile.to_widgets()

In [7]:
df.head(5)

,EBELN,EBELP,BSART,LIFNR,NAME1,EKORG,EKGRP,BEDAT,BSTYP,WERKS,...,PSTYP,MTART,ELIKZ,ERNAM,MENGE,NETPR,NETPR2,MEINS,WAERS,EMATN
0,4800052533,00001,ZB,0000007418,TRICON ENERGY LTD,PROD,MM5,20210629,F,PKTN,...,0,RAW,X,HAKHOURY,43857.24,405.75,4.0575,GAL,USD,000000000002099002
1,4800052534,00001,ZB,0000007418,TRICON ENERGY LTD,PROD,MM5,20210629,F,PKTN,...,0,RAW,X,HAKHOURY,836825.59,306.75,3.0675,GAL,USD,000000000002099002
2,4800052400,00002,ZB,0000007418,TRICON ENERGY LTD,PROD,MM5,20210609,F,PKTN,...,0,RAW,X,HAKHOURY,1764000.00,405.75,4.0575,GAL,USD,000000000002099002
3,4800052415,00002,ZB,0000007418,TRICON ENERGY LTD,PROD,MM5,20210615,F,PKTN,...,0,RAW,X,HAKHOURY,40443.06,405.75,4.0575,GAL,USD,000000000002099002
4,4800052577,00001,ZB,0000007418,TRICON ENERGY LTD,PROD,MM5,20210701,F,PKTN,...,0,RAW,X,HAKHOURY,840947.97,306.75,3.0675,GAL,USD,000000000002099002


### 3.2 Exploritory Data Analysis (EDA)

#### 3.2.1 Remove unnecessary columns

In [8]:
# Remove columns with high cardinality and with data that is not useful to modeling
df = df.drop(columns=['EBELP','BSART','BSTYP','WERKS','PSTYP','ELIKZ','MTART','ELIKZ','EMATN','EKORG','EKGRP'])
df.head(5)

,EBELN,LIFNR,NAME1,BEDAT,MATKL,NETWR,ERNAM,MENGE,NETPR,NETPR2,MEINS,WAERS
0,4800052533,0000007418,TRICON ENERGY LTD,20210629,N03001000,177950.75,HAKHOURY,43857.24,405.75,4.0575,GAL,USD
1,4800052534,0000007418,TRICON ENERGY LTD,20210629,N03001000,2566962.50,HAKHOURY,836825.59,306.75,3.0675,GAL,USD
2,4800052400,0000007418,TRICON ENERGY LTD,20210609,N03001000,7157430.00,HAKHOURY,1764000.00,405.75,4.0575,GAL,USD
3,4800052415,0000007418,TRICON ENERGY LTD,20210615,N03001000,164097.72,HAKHOURY,40443.06,405.75,4.0575,GAL,USD
4,4800052577,0000007418,TRICON ENERGY LTD,20210701,N03001000,2579607.90,HAKHOURY,840947.97,306.75,3.0675,GAL,USD


In [9]:
print("The data set is shaped as (rows, columns): ", df.shape)

The data set is shaped as (rows, columns):  (963, 12)


#### 3.2.2 Explore NETPR2 values to ensure they are all within normal range for Benzene with UoM USD/GAL

In [10]:
df.NETPR2.describe()

count    963.000000
mean       2.618766
std        2.737180
min        0.319500
25%        2.110000
50%        2.600000
75%        2.980000
max       84.452385
Name: NETPR2, dtype: float64

We see that there is at least one value for NETPR2 that is out of range with a value of $84/gal. We'll extract the EBELN number and the calculated cost per gallon for any rows > $10/gal

In [17]:
df.loc[df['NETPR2'] >=10, ['EBELN','NETPR2']]

,EBELN,NETPR2


Looking at the information for EBLEN = 4800049554 in SAP we find that it was 1 gallon on order with a value of $25,xxx.xx. The notes indicate that this was a true up. We don't anticipate very many of these to exist in the data so we will simply drop any values >$10/gal

In [12]:
ULim = 10
print("There were ", str(len(df[df['NETPR2'] > ULim])), "rows greater than", str(ULim), 'before processing')
df = df[df['NETPR2'] <= ULim]
print("There were ", str(len(df[df['NETPR2'] > ULim])), "rows greater than", str(ULim), 'after processing.') 
print('\nThe statistics of the remaining rows are:')
df.describe()

There were  1 rows greater than 10 before processing
There were  0 rows greater than 10 after processing.

The statistics of the remaining rows are:


,NETWR,MENGE,NETPR,NETPR2
count,9.620000e+02,9.620000e+02,9.620000e+02,962.000000
mean,2.440715e+06,7.949900e+05,1.860869e+04,2.533700
std,1.616944e+06,5.901480e+05,3.117756e+05,0.724011
min,3.200000e-01,1.000000e-01,9.000000e-01,0.319500
25%,1.525594e+06,4.172835e+05,2.340000e+00,2.110000
50%,2.207069e+06,8.400000e+05,3.790000e+00,2.600000
75%,2.788800e+06,9.967443e+05,2.780000e+04,2.980000
max,1.398600e+07,4.200000e+06,9.670303e+06,4.600000
